# Local Controller Noise Analysis (Take 2)

Required penalty energy $E_p$ vs RMS noise amplitude for selected correlation times $\tau_X$.

This notebook follows the exact method from `reference/3qubit_real_numbers_1f_nose.ipynb`:
- Uses `sesolve` (closed system) with OU noise on control fields
- Single realization per point (fixed seed for reproducibility)
- Sinusoidal RAP pulses
- Fidelity = projection onto |1_L⟩

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from qutip import tensor, sigmax, sigmaz, qeye, basis, sesolve
from qutip.coefficient import coefficient as from_array

sys.path.insert(0, os.path.dirname(os.getcwd()))
from qec_config import PlotConfig

PlotConfig.apply()

# Output directories
OUTPUT_DIR = Path('../figs/control_noise')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR = Path('../data/control_noise')
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Output: {OUTPUT_DIR}")
print(f"Data: {DATA_DIR}")

In [ ]:
# === 3-Qubit Repetition Code Operators ===
I, X, Z = qeye(2), sigmax(), sigmaz()
ket0, ket1 = basis(2, 0), basis(2, 1)

# Logical operators
X_L = tensor(X, X, X)
Z_L = tensor(Z, Z, Z)

# Stabilizers
S1 = tensor(Z, Z, I)
S2 = tensor(I, Z, Z)

# Single-qubit operators for noise
X1 = tensor(X, I, I)
X2 = tensor(I, X, I)
X3 = tensor(I, I, X)
Z1 = tensor(Z, I, I)
Z2 = tensor(I, Z, I)
Z3 = tensor(I, I, Z)

# Logical states
logical_zero = tensor(ket0, ket0, ket0)
logical_one = tensor(ket1, ket1, ket1)

# Projector onto |1_L>
proj_1L = logical_one * logical_one.dag()

print("3-qubit repetition code operators initialized")

In [ ]:
# === Bacon-Shor [[4,1,1,2]] Operators ===

# Logical operators
X_L_bs = tensor(X, I, X, I)  # X1 X3
Z_L_bs = tensor(Z, Z, I, I)  # Z1 Z2

# Gauge operators
G_bs = [
    tensor(X, X, I, I),  # X1 X2
    tensor(Z, I, Z, I),  # Z1 Z3
    tensor(I, I, X, X),  # X3 X4
    tensor(I, Z, I, Z),  # Z2 Z4
]

# Single-qubit operators
X1_bs = tensor(X, I, I, I)
X2_bs = tensor(I, X, I, I)
X3_bs = tensor(I, I, X, I)
X4_bs = tensor(I, I, I, X)
Z1_bs = tensor(Z, I, I, I)
Z2_bs = tensor(I, Z, I, I)
Z3_bs = tensor(I, I, Z, I)
Z4_bs = tensor(I, I, I, Z)

# Logical states
logical_zero_bs = (tensor(ket0, ket0, ket0, ket0) + tensor(ket1, ket1, ket1, ket1)).unit()
logical_one_bs = (X_L_bs * logical_zero_bs).unit()

# Projector onto |1_L>
proj_1L_bs = logical_one_bs * logical_one_bs.dag()

print("Bacon-Shor [[4,1,1,2]] operators initialized")

In [ ]:
# === Platform Parameters (IBM) ===
omega_max = 2 * np.pi * 12.5e6  # 12.5 MHz Rabi frequency (as in reference)
T_max = 4e-6                     # 4 us protocol duration
T_final = T_max

# Default noise parameters
rms_X_frac_default = 0.20  # 20% RMS
rms_Z_frac_default = 0.20  # 20% RMS  
tau_X_default = 0.15 * T_max
tau_Z_default = 0.15 * T_max

# Target fidelity
TARGET_FID = 0.99

# Ep grid (rad/s)
Ep_grid = np.linspace(1e4, 2 * np.pi * 150e6, 100)

# RMS amplitude sweep (as fraction of omega_max)
rms_X_fracs = np.linspace(0.1, 1.0, 10)  # 10% to 100%
rms_X_grid = rms_X_fracs * omega_max  # rad/s

# Correlation time grid
tau_X_grid = np.linspace(0.1 * T_max, 1.0 * T_max, 250)

# Fixed Z-noise parameters for the sweep
sigma_Z = 0.0  # Disable Z-noise for cleaner analysis
tau_Z = tau_Z_default

print(f"omega_max = {omega_max/(2*np.pi*1e6):.1f} MHz")
print(f"T_max = {T_max*1e6:.1f} us")
print(f"Ep_grid: {Ep_grid[0]/(2*np.pi*1e6):.2f} - {Ep_grid[-1]/(2*np.pi*1e6):.1f} MHz ({len(Ep_grid)} points)")
print(f"RMS range: {rms_X_fracs[0]*100:.0f}% - {rms_X_fracs[-1]*100:.0f}% ({len(rms_X_fracs)} points)")
print(f"tau_X range: {tau_X_grid[0]*1e6:.2f} - {tau_X_grid[-1]*1e6:.2f} us ({len(tau_X_grid)} points)")
print(f"Target fidelity: {TARGET_FID}")

In [ ]:
# === Noise Generation Functions ===

def sigma_from_rms(target_rms, tau):
    """Convert RMS to OU sigma: sigma = sqrt(2)*RMS/sqrt(tau)"""
    return (np.sqrt(2.0) * float(target_rms)) / np.sqrt(float(tau))

def ou_trace(times, tau, sigma, rng):
    """Generate Ornstein-Uhlenbeck process trace."""
    dt = np.diff(times, prepend=times[0])
    x = 0.0
    out = np.empty_like(times, dtype=float)
    for i, dti in enumerate(dt):
        x += (-x/tau)*dti + sigma*np.sqrt(max(dti, 1e-16))*rng.normal()
        out[i] = x
    return out

# RAP pulse functions (sinusoidal)
def omega_t(t):
    return omega_max * np.sin(np.pi * t / T_max)

def delta_t(t):
    return -omega_max * np.cos(np.pi * t / T_max)

print("Noise and pulse functions defined")

In [ ]:
# === Fidelity Computation Functions ===

def Ep_for_target_fid_rep(sigma_X_val, tau_X_val, Ep_grid, T_final, target_fid):
    """Find minimum Ep for target fidelity (3-qubit repetition code)."""
    tlist = np.linspace(0, T_final, 51)
    
    for Ep_val in Ep_grid:
        rng = np.random.default_rng(42)  # Fixed seed for reproducibility
        x1 = ou_trace(tlist, tau_X_val, sigma_X_val, rng)
        x2 = ou_trace(tlist, tau_X_val, sigma_X_val, rng)
        x3 = ou_trace(tlist, tau_X_val, sigma_X_val, rng)
        
        xiX = [from_array(tlist, x) for x in [x1, x2, x3]]
        
        H = [
            [X_L, lambda t, *args: omega_t(t)],
            [Z_L, lambda t, *args: delta_t(t)],
            -(Ep_val) * (S1 + S2),
            [X1, xiX[0]], [X2, xiX[1]], [X3, xiX[2]],
        ]
        
        sol = sesolve(H, logical_zero, tlist, e_ops=[proj_1L])
        final_fid = np.real(sol.expect[0][-1])
        
        if final_fid >= target_fid:
            return Ep_val
    
    return np.nan


def Ep_for_target_fid_bs(sigma_X_val, tau_X_val, Ep_grid, T_final, target_fid):
    """Find minimum Ep for target fidelity (Bacon-Shor code)."""
    tlist = np.linspace(0, T_final, 51)
    
    for Ep_val in Ep_grid:
        rng = np.random.default_rng(42)  # Fixed seed for reproducibility
        x1 = ou_trace(tlist, tau_X_val, sigma_X_val, rng)
        x2 = ou_trace(tlist, tau_X_val, sigma_X_val, rng)
        x3 = ou_trace(tlist, tau_X_val, sigma_X_val, rng)
        x4 = ou_trace(tlist, tau_X_val, sigma_X_val, rng)
        
        xiX = [from_array(tlist, x) for x in [x1, x2, x3, x4]]
        
        H = [
            [X_L_bs, lambda t, *args: omega_t(t)],
            [Z_L_bs, lambda t, *args: delta_t(t)],
            -(Ep_val) * sum(G_bs),
            [X1_bs, xiX[0]], [X2_bs, xiX[1]], [X3_bs, xiX[2]], [X4_bs, xiX[3]],
        ]
        
        sol = sesolve(H, logical_zero_bs, tlist, e_ops=[proj_1L_bs])
        final_fid = np.real(sol.expect[0][-1])
        
        if final_fid >= target_fid:
            return Ep_val
    
    return np.nan

print("Fidelity functions defined")

In [ ]:
# === Compute Ep map for Repetition Code ===
print("Computing Ep map for Repetition Code [[3,1,3]]...")
print("="*60)

Ep_map_rep = np.zeros((len(rms_X_grid), len(tau_X_grid)))

for j, tx in enumerate(tqdm(tau_X_grid, desc="tau_X sweep")):
    for i, rmsX in enumerate(rms_X_grid):
        sigmaX = sigma_from_rms(rmsX, tx)
        Ep_map_rep[i, j] = Ep_for_target_fid_rep(sigmaX, tx, Ep_grid, T_final, TARGET_FID)

print("="*60)
print("Done!")

In [ ]:
# === Compute Ep map for Bacon-Shor Code ===
print("Computing Ep map for Bacon-Shor [[4,1,1,2]]...")
print("="*60)

Ep_map_bs = np.zeros((len(rms_X_grid), len(tau_X_grid)))

for j, tx in enumerate(tqdm(tau_X_grid, desc="tau_X sweep")):
    for i, rmsX in enumerate(rms_X_grid):
        sigmaX = sigma_from_rms(rmsX, tx)
        Ep_map_bs[i, j] = Ep_for_target_fid_bs(sigmaX, tx, Ep_grid, T_final, TARGET_FID)

print("="*60)
print("Done!")

In [ ]:
# === Save computed data ===
data = {
    # Grids
    'rms_X_fracs': rms_X_fracs,
    'rms_X_grid': rms_X_grid,
    'tau_X_grid': tau_X_grid,
    'Ep_grid': Ep_grid,
    # Results
    'Ep_map_rep': Ep_map_rep,
    'Ep_map_bs': Ep_map_bs,
    # Parameters
    'omega_max': omega_max,
    'T_max': T_max,
    'target_fid': TARGET_FID,
    'sigma_Z': sigma_Z,
    'tau_Z': tau_Z,
}

np.savez_compressed(DATA_DIR / f'Ep_heatmap_RMS_vs_tau_X_F{TARGET_FID}_data.npz', **data)
print(f"Saved data to {DATA_DIR / f'Ep_heatmap_RMS_vs_tau_X_F{TARGET_FID}_data.npz'}")

In [ ]:
# === 1D Plot: Ep vs RMS Amplitude for selected tau_X values ===

# Select tau_X values to plot
n_tau_samples = 6
tau_indices = np.linspace(0, len(tau_X_grid)-1, n_tau_samples, dtype=int)

# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Color palette
colors = plt.cm.viridis(np.linspace(0.15, 0.85, n_tau_samples))

# Plot Repetition Code
for i, tau_idx in enumerate(tau_indices):
    tau_val = tau_X_grid[tau_idx]
    Ep_slice = Ep_map_rep[:, tau_idx]
    
    ax1.plot(rms_X_grid / (2*np.pi*1e6), Ep_slice / (2*np.pi*1e6),
             'o-', color=colors[i], linewidth=2, markersize=6,
             label=rf'$\tau_X$ = {tau_val*1e6:.2f} $\mu$s')

ax1.set_ylabel(r'Required $E_p$ [MHz]', fontsize=16)
ax1.set_title(r'(a) Repetition $[[3,1,3]]$', fontsize=18)
ax1.grid(True, alpha=0.3)
ax1.tick_params(labelsize=14)
ax1.legend(fontsize=10, loc='upper left')

# Plot Bacon-Shor Code
for i, tau_idx in enumerate(tau_indices):
    tau_val = tau_X_grid[tau_idx]
    Ep_slice = Ep_map_bs[:, tau_idx]
    
    ax2.plot(rms_X_grid / (2*np.pi*1e6), Ep_slice / (2*np.pi*1e6),
             's-', color=colors[i], linewidth=2, markersize=6,
             label=rf'$\tau_X$ = {tau_val*1e6:.2f} $\mu$s')

ax2.set_title(r'(b) Bacon-Shor $[[4,1,1,2]]$', fontsize=18)
ax2.grid(True, alpha=0.3)
ax2.tick_params(labelsize=14)
ax2.legend(fontsize=10, loc='upper left')

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)

# Shared x label
fig.text(0.5, 0.02, r'RMS Amplitude [MHz]', ha='center', fontsize=16)

plt.show()
print("Done: 1D plot of Ep vs RMS Amplitude for selected tau_X values.")

In [ ]:
# === Save figures ===
fig.savefig(OUTPUT_DIR / f'Ep_vs_RMS_comparison_F{TARGET_FID}.pdf', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / f'Ep_vs_RMS_comparison_F{TARGET_FID}.svg', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / f'Ep_vs_RMS_comparison_F{TARGET_FID}.png', dpi=300, bbox_inches='tight')

print(f"Saved figures to {OUTPUT_DIR}")
for f in sorted(OUTPUT_DIR.glob(f'Ep_vs_RMS*F{TARGET_FID}*')):
    print(f"  {f.name}")

In [ ]:
# === Summary ===
print("\n" + "="*70)
print(f"SUMMARY: Required Ep [MHz] for F >= {TARGET_FID}")
print("="*70)

# Select a few tau values for display
tau_display_indices = [0, len(tau_X_grid)//4, len(tau_X_grid)//2, -1]

print(f"\nRepetition Code [[3,1,3]]:")
print(f"{'RMS [MHz]':<12}", end="")
for idx in tau_display_indices:
    tau_us = tau_X_grid[idx] * 1e6
    print(f"  tau={tau_us:.2f}us", end="")
print()
print("-"*60)
for i, rmsX in enumerate(rms_X_grid):
    rms_MHz = rmsX / (2*np.pi*1e6)
    print(f"{rms_MHz:8.2f}    ", end="")
    for idx in tau_display_indices:
        Ep_MHz = Ep_map_rep[i, idx] / (2*np.pi*1e6)
        if np.isnan(Ep_MHz):
            print(f"      >150", end="")
        else:
            print(f"    {Ep_MHz:6.1f}", end="")
    print()

print(f"\nBacon-Shor Code [[4,1,1,2]]:")
print(f"{'RMS [MHz]':<12}", end="")
for idx in tau_display_indices:
    tau_us = tau_X_grid[idx] * 1e6
    print(f"  tau={tau_us:.2f}us", end="")
print()
print("-"*60)
for i, rmsX in enumerate(rms_X_grid):
    rms_MHz = rmsX / (2*np.pi*1e6)
    print(f"{rms_MHz:8.2f}    ", end="")
    for idx in tau_display_indices:
        Ep_MHz = Ep_map_bs[i, idx] / (2*np.pi*1e6)
        if np.isnan(Ep_MHz):
            print(f"      >150", end="")
        else:
            print(f"    {Ep_MHz:6.1f}", end="")
    print()

print("="*70)